In [337]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from scipy.stats import randint
import pickle
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import accuracy_score
# import packages for hyperparameters tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold, KFold, GridSearchCV
from sklearn.metrics import f1_score, roc_auc_score,accuracy_score,confusion_matrix, precision_recall_curve, auc, roc_curve, recall_score, classification_report 
from xgboost import XGBClassifier
import xgboost as xgb

from catboost import CatBoostClassifier

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.utils import resample,shuffle

from lightgbm import LGBMClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingClassifier
import time

In [338]:
train = pd.read_csv('data/train_clean_v4.csv')
valid = pd.read_csv('data/tmp_clean_v4.csv')

# 过采样
# train_big = train[train['Response']==0]
# train_small = train[train['Response']==1]
# train_small_up = resample(train_small,replace=True,n_samples=len(train_big),random_state = 0)
# train_balance = pd.concat([train_big, train_small_up])
# train_balance = shuffle(train_balance)
# train_balance.Response.value_counts()
train.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304888 entries, 0 to 304887
Data columns (total 26 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   id                                    304888 non-null  float64
 1   Age                                   304888 non-null  float64
 2   Driving_License                       304888 non-null  float64
 3   Region_Code                           304888 non-null  float64
 4   Previously_Insured                    304888 non-null  int64  
 5   Annual_Premium                        304888 non-null  float64
 6   Policy_Sales_Channel                  304888 non-null  float64
 7   Vintage                               304888 non-null  float64
 8   Response                              304888 non-null  int64  
 9   mean_Policy_Region_Age                304888 non-null  float64
 10  std_Policy_Region_Age                 304888 non-null  float64
 11  

In [339]:
train

,id,Age,Driving_License,Region_Code,Previously_Insured,Annual_Premium,Policy_Sales_Channel,Vintage,Response,mean_Policy_Region_Age,...,std_Policy_Region_Vintage,nunique_Policy_Region_Vintage,Gender_Male,Vehicle_Age_1_Year_down,Vehicle_Age_2_Years_up,Vehicle_Damage_Yes,PSC_Plus_1,PSC_Plus_2,PSC_Plus_3,RC_Plus_1
0,-1.732045,-0.762784,1.0,1.482552,1,0.150395,0.737734,0.820034,0,-1.159158,...,0.175606,0.461397,1,1,0,0,1,0,0,0
1,-1.732034,0.332966,1.0,0.121874,0,1.743285,-1.586528,0.043264,1,1.087133,...,0.059754,0.461397,0,0,1,1,0,1,0,1
2,-1.732022,0.848613,1.0,1.482552,0,-0.469641,0.221231,0.855885,1,0.660620,...,0.073876,0.461397,1,0,1,1,0,0,1,0
3,-1.732011,-1.020608,1.0,1.482552,1,-1.623761,0.737734,0.103016,0,-1.159158,...,0.175606,0.461397,1,1,0,0,1,0,0,0
4,-1.732000,0.268510,1.0,0.121874,0,0.115793,0.221231,-1.175668,1,0.702061,...,-0.003576,0.461397,0,0,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304883,1.732000,-1.213975,1.0,-1.465583,0,-1.623761,0.885306,1.309997,0,-1.266242,...,0.037181,-1.310622,0,1,0,1,0,0,0,0
304884,1.732011,-0.956152,1.0,-0.482871,0,-1.623761,-1.586528,-1.092016,0,1.186185,...,0.004700,-0.220149,0,1,0,1,0,1,0,0
304885,1.732022,-0.891696,1.0,0.726620,1,-0.216672,0.737734,-1.534178,0,-1.129398,...,0.186608,0.461397,0,1,0,0,1,0,0,0
304886,1.732034,-1.085064,1.0,-1.389990,1,2.384081,0.737734,-0.171842,0,-1.108251,...,0.070221,0.461397,1,1,0,0,1,0,0,0


In [340]:
valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76221 entries, 0 to 76220
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    76221 non-null  float64
 1   Age                                   76221 non-null  float64
 2   Driving_License                       76221 non-null  float64
 3   Region_Code                           76221 non-null  float64
 4   Previously_Insured                    76221 non-null  int64  
 5   Annual_Premium                        76221 non-null  float64
 6   Policy_Sales_Channel                  76221 non-null  float64
 7   Vintage                               76221 non-null  float64
 8   Response                              76221 non-null  int64  
 9   mean_Policy_Region_Age                76221 non-null  float64
 10  std_Policy_Region_Age                 76221 non-null  float64
 11  nunique_Policy_

In [341]:
train_target = train['Response']
train.drop(['Response','id'], axis=1, inplace=True)
valid_target = valid['Response']
valid_id = valid['id']
valid.drop(['Response','id'], axis=1, inplace=True)

# ## 标椎化
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# train = scaler.fit_transform(train)
# scaler = MinMaxScaler()
# valid = scaler.fit_transform(valid)

train_X = train
train_Y = train_target
# train_X, test_X, train_Y, test_Y = train_test_split(train, train_target, test_size=0.001)

In [342]:
train_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304888 entries, 0 to 304887
Data columns (total 24 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   Age                                   304888 non-null  float64
 1   Driving_License                       304888 non-null  float64
 2   Region_Code                           304888 non-null  float64
 3   Previously_Insured                    304888 non-null  int64  
 4   Annual_Premium                        304888 non-null  float64
 5   Policy_Sales_Channel                  304888 non-null  float64
 6   Vintage                               304888 non-null  float64
 7   mean_Policy_Region_Age                304888 non-null  float64
 8   std_Policy_Region_Age                 304888 non-null  float64
 9   nunique_Policy_Region_Age             304888 non-null  float64
 10  mean_Policy_Region_Annual_Premium     304888 non-null  float64
 11  

## 测试多项式升维

In [285]:
poly = PolynomialFeatures(degree=2,include_bias=False)
train_X_plus = poly.fit_transform(train_X)
test_X_plus = poly.fit_transform(test_X)
valid_plus = poly.fit_transform(valid)

print("新特征数量：", train_X_plus.shape[1])

新特征数量： 324


In [289]:
# 特征选择
time_start = time.time()
clf=XGBClassifier(n_estimators = 300, max_depth = 13, gamma = 2.3408807913619945, reg_lambda = 0.3770436657913232,reg_alpha = 40.0, min_child_weight=7.0,colsample_bytree = 0.5786479102658189 ,random_state = 0).fit(train_X_plus, train_Y)

# clf = GradientBoostingClassifier().fit(train_X_plus, train_Y)
print("feature importance after trainning:", clf.feature_importances_)
print("totoal trainning leghth:", time.time() - time_start)


feature importance after trainning: [1.92628161e-03 0.00000000e+00 3.15741636e-04 1.13469966e-01
 2.01634466e-04 8.10272584e-04 2.59828201e-04 2.20122677e-03
 2.54086335e-04 7.20848388e-04 2.96532962e-04 5.20945701e-04
 3.09039548e-04 2.55968072e-04 3.36717145e-04 3.35586257e-04
 2.93065968e-04 3.38886428e-04 0.00000000e+00 1.38465194e-02
 2.89642019e-04 0.00000000e+00 0.00000000e+00 0.00000000e+00
 2.44681747e-03 2.24503456e-04 2.27203520e-04 0.00000000e+00
 1.92076637e-04 3.41493054e-04 1.94913373e-04 3.39062914e-04
 1.30285334e-03 2.19209367e-04 1.96489156e-04 1.98210313e-04
 2.14475731e-04 1.73993074e-04 2.05435092e-04 2.13042164e-04
 2.30928927e-04 1.12162139e-02 2.14991931e-04 1.42438163e-03
 2.61072128e-04 6.28482143e-04 6.37377962e-04 3.15661542e-04
 0.00000000e+00 4.45439975e-04 2.03621183e-02 1.80574076e-04
 2.65050447e-04 1.87685611e-04 1.27343903e-03 2.12463536e-04
 1.74383866e-04 2.58758169e-04 2.56584754e-04 2.72946345e-04
 2.73503741e-04 2.55576480e-04 1.73572276e-04 1.1

In [298]:
sel_model = SelectFromModel(clf, prefit=True, threshold=-np.inf, max_features=50)
train_X_2 = sel_model.transform(train_X_plus)
test_X_2 = sel_model.transform(test_X_plus)
valid_2 = sel_model.transform(valid_plus)

print("新的特征数量:", train_X_2.shape[1])

新的特征数量: 50


In [ ]:
# ## pca 降维
# from sklearn.decomposition import PCA

# pca = PCA(n_components=0.98)

# pca.fit(train_X_plus)
# train_X_pca = pca.transform(train_X_plus)
# test_X_pca = pca.transform(test_X_plus)
# valid_pca = pca.transform(valid_plus)

In [343]:
XGB_=XGBClassifier(n_estimators = 300, max_depth = 7, gamma = 2.5952033716242866, reg_lambda = 0.707680330032109,reg_alpha = 40.0, min_child_weight=7.0,colsample_bytree = 0.6611082911056974,random_state = 0,n_jobs=-1)

# XGB_ = XGBClassifier(n_estimators=1000,max_depth=6,learning_rate=0.04,subsample=0.9,colsample_bytree=0.35,objective = 'binary:logistic',random_state = 1) 
time_start = time.time()
XGB_fit = XGB_.fit(train_X, train_Y)
print("totoal trainning leghth:", time.time() - time_start)

totoal trainning leghth: 96.52957582473755


In [300]:
CAB = CatBoostClassifier(learning_rate = 0.055,random_state=42,scale_pos_weight=7, custom_metric=['AUC'])

# XGB_ = XGBClassifier(n_estimators=1000,max_depth=6,learning_rate=0.04,subsample=0.9,colsample_bytree=0.35,objective = 'binary:logistic',random_state = 1) 
time_start = time.time()
CAB_fit = CAB.fit(train_X, train_Y)
print("totoal trainning leghth:", time.time() - time_start)

: 24s	remaining: 15.1s
614:	learn: 0.4088840	total: 24s	remaining: 15s
615:	learn: 0.4088636	total: 24.1s	remaining: 15s
616:	learn: 0.4088454	total: 24.1s	remaining: 15s
617:	learn: 0.4088270	total: 24.1s	remaining: 14.9s
618:	learn: 0.4088133	total: 24.2s	remaining: 14.9s
619:	learn: 0.4087862	total: 24.2s	remaining: 14.8s
620:	learn: 0.4087701	total: 24.3s	remaining: 14.8s
621:	learn: 0.4087547	total: 24.3s	remaining: 14.8s
622:	learn: 0.4087383	total: 24.3s	remaining: 14.7s
623:	learn: 0.4087187	total: 24.4s	remaining: 14.7s
624:	learn: 0.4087162	total: 24.4s	remaining: 14.7s
625:	learn: 0.4087088	total: 24.5s	remaining: 14.6s
626:	learn: 0.4086919	total: 24.5s	remaining: 14.6s
627:	learn: 0.4086750	total: 24.5s	remaining: 14.5s
628:	learn: 0.4086597	total: 24.6s	remaining: 14.5s
629:	learn: 0.4086457	total: 24.6s	remaining: 14.5s
630:	learn: 0.4086355	total: 24.7s	remaining: 14.4s
631:	learn: 0.4086170	total: 24.7s	remaining: 14.4s
632:	learn: 0.4086039	total: 24.7s	remaining: 14.

In [344]:
lg=LGBMClassifier(boosting_type='gbdt',n_estimators=500,depth=10,learning_rate=0.04,objective='binary',metric='auc',colsample_bytree=0.5,reg_lambda=2,reg_alpha=2,random_state=294,n_jobs=-1) 
# lg = LGBMClassifier(boosting_type='gbdt',n_estimators=10000,max_depth=10,learning_rate=0.02,subsample=0.9,colsample_bytree=0.4,objective ='binary',random_state = 1,importance_type='gain',reg_alpha=2,reg_lambda=2)

time_start = time.time()
lg_fit = lg.fit(train_X, train_Y)
print("totoal trainning leghth:", time.time() - time_start)

[LightGBM] [Warning] Unknown parameter: depth
totoal trainning leghth: 14.284417629241943


In [361]:
yscore = XGB_fit.predict_proba(valid)[:,1]
lg_yscore = lg_fit.predict_proba(valid)[:,1]
best_auc = 0.
best_x = 0.
for x in np.arange(0,1.001,0.001):
    fpr, tpr, _ = roc_curve(valid_target, yscore * x + lg_yscore * (1-x))
    tmp_auc = auc(fpr,tpr)
    if tmp_auc > best_auc:
        best_auc = tmp_auc
        best_x = x

# plt.title('XGBOOST ROC curve')
# plt.xlabel('FPR (Precision)')
# plt.ylabel('TPR (Recall)')

# plt.plot(fpr,tpr)
# plt.plot((0,1), ls='dashed',color='black')
# plt.show()
print ('Area under curve (AUC): ', format(round(best_auc,5)))
print ('Best X:', format(round(best_x,5)))



Area under curve (AUC):  0.86045
Best X: 0.248


In [351]:
valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76221 entries, 0 to 76220
Data columns (total 24 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Age                                   76221 non-null  float64
 1   Driving_License                       76221 non-null  float64
 2   Region_Code                           76221 non-null  float64
 3   Previously_Insured                    76221 non-null  int64  
 4   Annual_Premium                        76221 non-null  float64
 5   Policy_Sales_Channel                  76221 non-null  float64
 6   Vintage                               76221 non-null  float64
 7   mean_Policy_Region_Age                76221 non-null  float64
 8   std_Policy_Region_Age                 76221 non-null  float64
 9   nunique_Policy_Region_Age             76221 non-null  float64
 10  mean_Policy_Region_Annual_Premium     76221 non-null  float64
 11  std_Policy_Regi

In [362]:
df_ans = pd.DataFrame(range(76221),columns=['id'])
df_ans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76221 entries, 0 to 76220
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      76221 non-null  int64
dtypes: int64(1)
memory usage: 595.6 KB


In [363]:
df_ans['Response'] = yscore * 0.25 + lg_yscore * (1-0.25)
df_ans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76221 entries, 0 to 76220
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        76221 non-null  int64  
 1   Response  76221 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 1.2 MB


# ANS!!!!!!

In [364]:
df_ans.to_csv('data/ans.csv') 

In [327]:
yscore = XGB_fit.predict_proba(test_X)[:,1]
lg_yscore = lg_fit.predict_proba(test_X)[:,1]
best_auc = 0.
best_x = 0.
for x in np.arange(0,1.01,0.01):
    fpr, tpr, _ = roc_curve(test_Y, yscore * x + lg_yscore * (1-x))
    tmp_auc = auc(fpr,tpr)
    if tmp_auc > best_auc:
        best_auc = tmp_auc
        best_x = x

# plt.title('XGBOOST ROC curve')
# plt.xlabel('FPR (Precision)')
# plt.ylabel('TPR (Recall)')

# plt.plot(fpr,tpr)
# plt.plot((0,1), ls='dashed',color='black')
# plt.show()
print ('Area under curve (AUC): ', format(round(best_auc,5)))
print ('Best X:', format(round(best_x,5)))

Area under curve (AUC):  0.85788
Best X: 0.54


In [ ]:
yscore = XGB_fit.predict_proba(train_X)[:,1]
lg_yscore = lg_fit.predict_proba(train_X)[:,1]
best_auc = 0.
best_x = 0.
for x in np.arange(0,1.01,0.01):
    fpr, tpr, _ = roc_curve(train_Y, yscore * x + lg_yscore * (1-x))
    tmp_auc = auc(fpr,tpr)
    if tmp_auc > best_auc:
        best_auc = tmp_auc
        best_x = x

# plt.title('XGBOOST ROC curve')
# plt.xlabel('FPR (Precision)')
# plt.ylabel('TPR (Recall)')

# plt.plot(fpr,tpr)
# plt.plot((0,1), ls='dashed',color='black')
# plt.show()
print ('Area under curve (AUC): ', format(round(best_auc,5)))
print ('Best X:', format(round(best_x,5)))

In [ ]:
# random_search = {'criterion': ['entropy', 'gini'],
#                'max_depth': [2,3,4,5,6,7,10],
#                'min_samples_leaf': [4, 6, 8],
#                'min_samples_split': [5, 7,10],
#                'n_estimators': [300]}

# clf = RandomForestClassifier()
# model = RandomizedSearchCV(estimator = clf, param_distributions = random_search, n_iter = 10, 
#                                cv = 4, verbose= 1, random_state= 101, n_jobs = -1)
# model.fit(train, train_target)

In [ ]:
# model

In [ ]:
# y_pred=model.predict(test)
# print (classification_report(test_target, y_pred))

In [ ]:
# sns.countplot(train_target)


In [ ]:
# y_score = model.predict_proba(test)[:,1]
# fpr, tpr, _ = roc_curve(test_target, y_score)

# plt.title('Random Forest ROC curve: CC Fraud')
# plt.xlabel('FPR (Precision)')
# plt.ylabel('TPR , y_pred')

# plt.plot(fpr,tpr)
# plt.plot((0,1), ls='dashed',color='black')
# plt.show()
# print ('Area under curve (AUC): ', auc(fpr,tpr))

In [ ]:
model.best_estimator_

In [ ]:
pickle.dump(model, open('rf_model_clean_v1.sav', 'wb'))

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [ ]:
train_svc = train.copy()
train_svc_target = train_target.copy()

train_X, test_X, train_y, test_y = train_test_split(train_svc, train_svc_target, test_size=0.2, random_state=42)

In [ ]:
train_svc_X = train_X[:10000]
train_svc_Y = train_y[:10000]

In [ ]:
random_search_svc = {'C': 10. ** np.arange(-3, 8),
               'gamma' : 10. ** np.arange(-5, 4),}
clf_svc = SVC(probability=True)
svc_model = RandomizedSearchCV(estimator = clf_svc, param_distributions = random_search_svc, n_jobs\
                                         = -1, cv=5)

svc_model.fit(train_svc_X, train_svc_Y)

In [ ]:
svc_model.best_params_

In [ ]:
y_score2 = svc_model.predict_proba(test_X)[:,1]
fpr, tpr, _ = roc_curve(test_y, y_score2)

plt.title('SVC ROC curve: CC Fraud')
plt.xlabel('FPR (Precision)')
plt.ylabel('TPR , y_pred')

plt.plot(fpr,tpr)
plt.plot((0,1), ls='dashed',color='black')
plt.show()
print ('Area under curve (AUC): ', auc(fpr,tpr))

In [ ]:
y_score2 = svc_model.predict_proba(test)[:,1]
fpr, tpr, _ = roc_curve(test_target, y_score2)

plt.title('SVC ROC curve: CC Fraud')
plt.xlabel('FPR (Precision)')
plt.ylabel('TPR , y_pred')

plt.plot(fpr,tpr)
plt.plot((0,1), ls='dashed',color='black')
plt.show()
print ('Area under curve (AUC): ', auc(fpr,tpr))

In [ ]:
pickle.dump(svc_model, open('model/svc_model_clean_v1.sav', 'wb'))

In [ ]:
y_score2

In [ ]:
train

In [ ]:
test_y.sum()

In [ ]:
clf = xgb.XGBClassifier()


param_grid = {
        'silent': [False],
        'max_depth': [6, 10, 15, 20],
        'learning_rate': [0.001, 0.01, 0.1, 0.2, 0,3],
        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
        'gamma': [0, 0.25, 0.5, 1.0],
        'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
        'n_estimators': [100]}

fit_params = {'eval_metric': 'mlogloss',
              'early_stopping_rounds': 10,
              'eval_set': [(test_X, test_y)]}

rs_xgb_model = RandomizedSearchCV(clf, param_grid, n_iter=20,
                            n_jobs=-1, verbose=2, cv=2,
                            scoring='neg_log_loss', random_state=42)

In [ ]:
import time

In [ ]:
search_time_start = time.time()
rs_xgb_model.fit(train_X, train_y)
print("Randomized search time:", time.time() - search_time_start)


In [ ]:
rs_xgb_model.best_params_

In [ ]:
train_balance_target = train_balance['Response']
train_balance.drop(['Response'], axis=1, inplace=True)
valid_target = valid['Response']
valid.drop(['Response'], axis=1, inplace=True)
train_X, test_X, train_Y, test_Y = train_test_split(train_balance, train_balance_target, test_size=0.33)

In [ ]:
clf2 = xgb.XGBClassifier()

param_dist = {
        'n_estimators':[300],
        'max_depth':range(2,15,1),
        'learning_rate':np.linspace(0.01,2,20),
        'subsample':np.linspace(0.7,0.9,20),
        'colsample_bytree':np.linspace(0.5,0.98,10),
        'min_child_weight':range(1,9,1) }

rs_xgb_model_v2 = RandomizedSearchCV(clf2, param_dist, n_iter=300, \
                        n_jobs=-1, cv=3, verbose=1, scoring='neg_log_loss')

In [ ]:
test_Y

In [ ]:
import time

In [ ]:
XGB_=XGBClassifier(n_estimators = 300, max_depth = 13, gamma = 2.3408807913619945, reg_lambda = 0.3770436657913232,
                            reg_alpha = 40.0, min_child_weight=7.0,colsample_bytree = 0.5786479102658189 ,random_state = 0)
time_start = time.time()
XGB_fit = XGB_.fit(train_X_2, train_Y)
print("totoal trainning leghth:", time.time() - time_start)

In [ ]:
## 过采样

yscore = XGB_fit.predict_proba(valid)[:,1]
fpr, tpr, _ = roc_curve(valid_target, yscore)

plt.title('XGBOOST ROC curve')
plt.xlabel('FPR (Precision)')
plt.ylabel('TPR (Recall)')

plt.plot(fpr,tpr)
plt.plot((0,1), ls='dashed',color='black')
plt.show()
print ('Area under curve (AUC): ', format(round(auc(fpr,tpr),5)))

In [ ]:
yscore = XGB_fit.predict_proba(valid_2)[:,1]
fpr, tpr, _ = roc_curve(valid_target, yscore)

plt.title('XGBOOST ROC curve')
plt.xlabel('FPR (Precision)')
plt.ylabel('TPR (Recall)')

plt.plot(fpr,tpr)
plt.plot((0,1), ls='dashed',color='black')
plt.show()
print ('Area under curve (AUC): ', format(round(auc(fpr,tpr),5)))

In [ ]:
yscore = XGB_fit.predict_proba(test_X_2)[:,1]
fpr, tpr, _ = roc_curve(test_Y, yscore)

plt.title('XGBOOST ROC curve')
plt.xlabel('FPR (Precision)')
plt.ylabel('TPR (Recall)')

plt.plot(fpr,tpr)
plt.plot((0,1), ls='dashed',color='black')
plt.show()
print ('Area under curve (AUC): ', format(round(auc(fpr,tpr),5)))

In [ ]:
##best param of us
XGB_=XGBClassifier(n_estimators = 300, max_depth = 7, gamma = 2.5952033716242866, reg_lambda = 0.707680330032109,
                            reg_alpha = 40.0, min_child_weight=7.0,colsample_bytree = 0.6611082911056974,random_state = 0)
time_start = time.time()
XGB_fit = XGB_.fit(train_X_2, train_Y)
print("totoal trainning leghth:", time.time() - time_start)

In [ ]:
yscore = XGB_fit.predict_proba(valid_2)[:,1]
fpr, tpr, _ = roc_curve(valid_target, yscore)

plt.title('XGBOOST ROC curve')
plt.xlabel('FPR (Precision)')
plt.ylabel('TPR (Recal l)')

plt.plot(fpr,tpr)
plt.plot((0,1), ls='dashed',color='black')
plt.show()
print ('Area under curve (AUC): ', format(round(auc(fpr,tpr),5)))

In [ ]:
yscore = XGB_fit.predict_proba(train_X_2)[:,1]
fpr, tpr, _ = roc_curve(train_Y, yscore)

plt.title('XGBOOST ROC curve')
plt.xlabel('FPR (Precision)')
plt.ylabel('TPR (Recal l)')

plt.plot(fpr,tpr)
plt.plot((0,1), ls='dashed',color='black')
plt.show()
print ('Area under curve (AUC): ', format(round(auc(fpr,tpr),5)))

In [ ]:
yscore = XGB_fit.predict_proba(test_X_2)[:,1]
fpr, tpr, _ = roc_curve(test_Y, yscore)

plt.title('XGBOOST ROC curve')
plt.xlabel('FPR (Precision)')
plt.ylabel('TPR (Recal l)')

plt.plot(fpr,tpr)
plt.plot((0,1), ls='dashed',color='black')
plt.show()
print ('Area under curve (AUC): ', format(round(auc(fpr,tpr),5)))

In [ ]:
clf = xgb.XGBClassifier()


param_grid = {
        'silent': [False],
        'max_depth': [6, 10, 15, 20],
        'learning_rate': [0.001, 0.01, 0.1, 0.2, 0,3],
        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
        'gamma': [0, 0.25, 0.5, 1.0],
        'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
        'n_estimators': [300]}

fit_params = {'eval_metric': 'mlogloss',
              'early_stopping_rounds': 10,
              'eval_set': [(test_X, test_Y)]}

rs_xgb_model = RandomizedSearchCV(clf, param_grid, n_iter=20,
                            n_jobs=-1, verbose=2, cv=2,
                            scoring='neg_log_loss', random_state=42)

In [ ]:
import time

In [ ]:
search_time_start = time.time()
rs_xgb_model.fit(train_X, train_Y)
print("Randomized search time:", time.time() - search_time_start)
